## Handwritten digit recognition with MNIST using IBM Watson Studio Jupyter Notebook in Python & Watson Machine Learning capabilities & Toolbox (ART) - for Data Scientist.
This tutorial shows IBM Watson Studio framework capabilities to create Jupyter notebook leveraging Keras ML framework to create a convolutional neural network (CNN) model that is build & trained & improved in term of robustness & accuracy with IBM Watson Machine Learning Toolbox ART & IBM Watson ML capabilities.



This tutorial will leverage **IBM ART** *(Adversarial Robustness Toolbox)* open source library
available at https://github.com/IBM/adversarial-robustness-toolbox

This is a library dedicated to adversarial machine learning.
Its purpose is to allow rapid crafting and analysis of attacks and defense methods for machine learning models.
The Adversarial Robustness Toolbox provides an implementation for many state-of-the-art methods for attacking and defending classifiers.

The ART toolbox is developed with the goal of helping developers better understand:
 * Measuring model robustness
 * Model hardening
 * Runtime detection

For more information you can read `Adversarial Robustness Toolbox v0.3.0` IBM research publication from Nicolae, Maria-Irina and Sinn, Mathieu and Tran, Minh~Ngoc and Rawat, Ambrish and Wistuba, Martin and Zantedeschi, Valentina and Baracaldo, Nathalie and Chen, Bryant and Ludwig, Heiko and Molloy, Ian and Edwards, Ben
available here: https://arxiv.org/pdf/1807.01069

## Environment Setup 
Install base Data Science libraries, then Keras and ART into our environment and import all required classes.

In [ ]:
# Import base Data Science libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=100, facecolor='w', edgecolor='k')
%matplotlib inline

# Import Keras
import keras
print("Keras version :",keras.__version__)
from keras.models import load_model
import keras.backend as k
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras.optimizers import RMSprop

import warnings
warnings.resetwarnings() # Maybe somebody else is messing with the warnings system?
warnings.filterwarnings('ignore') # Ignore everything

# Install ART and import packages 
!pip install adversarial-robustness-toolbox
from art.utils import load_dataset
from art.classifiers import KerasClassifier
from art.attacks.fast_gradient import FastGradientMethod
from art.attacks.newtonfool import NewtonFool
from art.attacks.iterative_method import BasicIterativeMethod
from art.defences.adversarial_trainer import AdversarialTrainer

## Optional: set random seeds for reproducibility
#np.random.seed(99)
#import tensorflow as tf
#tf.set_random_seed(42);

### Define utility functions 
Utility functions to manipulate images

In [ ]:
# Define a few utility functions

def setupSubPlots(numCols,maxRows):
    numRows=1+(maxRows-1)//numCols
    return plt.subplots(numRows,numCols,squeeze=False,figsize=(20,numRows/3*4))

def plotImage(img,axs,ix,numCols,title=None):
    ax=axs[ix//numCols][ix%numCols]
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if title: 
        ax.set_title(title)
        ax.set_xticks([])
    return ax.imshow(img.squeeze())

def showImages(imgTable, limit, numCols=15, legend=None):
    ''' Image plotting function, using a set number of columns
    '''
    limit=min(limit,len(imgTable))
    if limit<numCols: numCols=limit
    fig, axs = setupSubPlots(numCols,limit)
    for ix,img in enumerate(imgTable):
        if ix>=limit: break
        plotImage(img,axs,ix,numCols)
    if legend:
        fig.suptitle(legend, fontsize=16)
    return

def showImagesAccuracy(description,images, labels, predictions, maxShown=30, numCols=15):
    ''' Check Images prediction accuracy, count and display mismatches
    '''
    idx = 0
    errorsCount = 0
    shown = 0
    fig, axs = setupSubPlots(numCols,maxShown)
    for idx,img in enumerate(images):
        predicted = np.argmax(predictions[idx])
        actual = np.argmax(labels[idx])
        if predicted != actual:
            errorsCount += 1
            if shown < maxShown:
                # Plotting first samples of MNIST
                plotImage(img,axs,shown,numCols,"{}->{}".format(actual,predicted))
                shown += 1
    # Compute accuracy as a percentage         
    accuracy = (100-(errorsCount/len(images))*100)
    legend="{}\nSucccess rate: {:.2f}%".format(description,accuracy)
    print(legend)
    #fig.suptitle(legend)
    return accuracy

## MNIST data set utility loading code
This code is used to load the MNISt dataset and prepare it for Keras

In [ ]:
def get_file1(filename, url, path=None, extract=False):
    """
    Downloads a file from a URL if it not already in the cache. The file at indicated by `url` is downloaded to the
    path `path` (default is ~/.art/data). and given the name `filename`. Files in tar, tar.gz, tar.bz, and zip formats
    can also be extracted. This is a simplified version of the function with the same name in Keras.

    :param filename: Name of the file.
    :type filename: `str`
    :param url: Download URL.
    :type url: `str`
    :param path: Folder to store the download. If not specified, `~/.art/data` is used instead.
    :type: `str`
    :param extract: If true, tries to extract the archive.
    :type extract: `bool`
    :return: Path to the downloaded file.
    :rtype: `str`
    """
    import os
    if path is None:
        from art import DATA_PATH
        path_ = os.path.expanduser(DATA_PATH)
    else:
        path_ = os.path.expanduser(path)
    if not os.access(path_, os.W_OK):
        path_ = os.path.join('./', '.art')
    if not os.path.exists(path_):
        os.makedirs(path_)

    if extract:
        extract_path = os.path.join(path_, filename)
        full_path = extract_path + '.tar.gz'
    else:
        full_path = os.path.join(path_, filename)

    # Determine if dataset needs downloading
    download = not os.path.exists(full_path)

    if download:
        # logger.info('Downloading data from %s', url)
        error_msg = 'URL fetch failure on {}: {} -- {}'
        try:
            try:
                from six.moves.urllib.error import HTTPError, URLError
                from six.moves.urllib.request import urlretrieve

                urlretrieve(url, full_path)
            except HTTPError as e:
                raise Exception(error_msg.format(url, e.code, e.msg))
            except URLError as e:
                raise Exception(error_msg.format(url, e.errno, e.reason))
        except (Exception, KeyboardInterrupt):
            if os.path.exists(full_path):
                os.remove(full_path)
            raise

    if extract:
        if not os.path.exists(extract_path):
            _extract(full_path, path_)
        return extract_path

    return full_path


def preprocess(x, y, nb_classes=10, max_value=255):
    """Scales `x` to [0, 1] and converts `y` to class categorical confidences.

    :param x: Data instances
    :type x: `np.ndarray`
    :param y: Labels
    :type y: `np.ndarray`
    :param nb_classes: Number of classes in dataset
    :type nb_classes: `int`
    :param max_value: Original maximum allowed value for features
    :type max_value: `int`
    :return: rescaled values of `x`, `y`
    :rtype: `tuple`
    """
    x = x.astype('float32') / max_value
    y = to_categorical(y, nb_classes)

    return x, y

def to_categorical(labels, nb_classes=None):
    """
    Convert an array of labels to binary class matrix.

    :param labels: An array of integer labels of shape `(nb_samples,)`
    :type labels: `np.ndarray`
    :param nb_classes: The number of classes (possible labels)
    :type nb_classes: `int`
    :return: A binary matrix representation of `y` in the shape `(nb_samples, nb_classes)`
    :rtype: `np.ndarray`
    """
    labels = np.array(labels, dtype=np.int32)
    if not nb_classes:
        nb_classes = np.max(labels) + 1
    categorical = np.zeros((labels.shape[0], nb_classes), dtype=np.float32)
    categorical[np.arange(labels.shape[0]), np.squeeze(labels)] = 1
    return categorical

def load_mnist_data(raw=False):
    """Loads MNIST dataset from `DATA_PATH` or downloads it if necessary.

    :param raw: `True` if no preprocessing should be applied to the data. Otherwise, data is normalized to 1.
    :type raw: `bool`
    :return: `(x_train, y_train), (x_test, y_test), min, max`
    :rtype: `(np.ndarray, np.ndarray), (np.ndarray, np.ndarray), float, float`
    """
    # from art import DATA_PATH
    
    path = get_file1('mnist.npz', url='https://s3.amazonaws.com/img-datasets/mnist.npz')

    f = np.load(path)
    x_train = f['x_train']
    y_train = f['y_train']
    x_test = f['x_test']
    y_test = f['y_test']
    f.close()

    # Add channel axis
    min_, max_ = 0, 255
    if not raw:
        min_, max_ = 0., 1.
        x_train = np.expand_dims(x_train, axis=3)
        x_test = np.expand_dims(x_test, axis=3)
        x_train, y_train = preprocess(x_train, y_train)
        x_test, y_test = preprocess(x_test, y_test)

    return (x_train, y_train), (x_test, y_test), min_, max_


##  Step 2 Load and prepare dataset for training
Load pre-shuffled **MNIST** (*Modified National Institute of Standards and Technology*) annotated data into train and test datasets and create our CNN model to be trained.

In [ ]:
# Read MNIST dataset
#(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset(str('mnist'))
(x_train, y_train), (x_test, y_test), min_, max_ = load_mnist_data()
print("There are {} images in the training set, and {} in the test set".format(len(x_train),len(x_test)))

### Show the first 30 images on 15 columns
We use one the functions defined above to get an idea of how the images from the MNIST dataset look like

In [ ]:
# Show first thirty images from the training set
showImages(x_train,30,15,"Example: first 30 original images from Training Set")

## Step3: Create Keras convolutional neural network
This uses a basic CNN architecture from the Keras examples.
The model layers are added sequentially:

**[Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)**
    It is a 2D convolutional layer that we use to process the 2D MNIST input images:
    * The first argument passed to the Conv2D() layer function is the number of output channels – in this case we have 32 output channels. 
    * `kernel_size`, is the size of the sampling moving window, here a 3x3 aperture
    * the activation function is a rectified linear unit (ReLU) 
    * The shape (size: height, width, color depth) of the input is inferred from the training set image size.
    Noe that declaring the input shape is only required of the first layer, Keras will then work out the size of the tensors flowing through the model (automatic shape inference). We start with 26x26 pixels and 32 bit colors

**[MaxPool2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D)**:
    Reduction matrix, here we halve each dimension, resulting shape will be a 13x13x32 matrix

**[Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten)**:
    Transform the input's shape n*c*h*w into a n*(c*h*w) vector) yields 13*13*32=5408 outputs as a flat vector

**[Dense(128,relu)](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)**:
    A layer where each neuron is linked to each ones of the next layer. This layer is set to yield 128 outputs

**[DropOut](https://www.tensorflow.org/api_docs/python/tf/keras/layers/DropOut)**:
    Freeze some neurons to avoid overfitting. *Dropout consists in randomly setting a fraction rate of input units to 0 at each update during training time, which helps prevent overfitting.*

**[Dense(10,SoftMax)]**
    Layer to transform a scoring distribution into a probability distribution, with 10 outputs for our 10 digits. *The [softmax](https://www.tensorflow.org/api_docs/python/tf/keras/activations/softmax) function normalizes ('squashes') a K-dimensional vector of real values to a K-dimensional vector of real values, where each entry is in the range (0, 1),[a] and all the entries add up to 1.

**Optimizer parameters**:
    * lr: Learning rate, interval used for the gradient descent algorithm
    * **[loss]()**: Cost/loss function used for this neural network, here we use *Categorical cross entropy*

In [ ]:
# Create Keras convolutional neural network - basic architecture from Keras examples
k.set_learning_phase(1)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=x_train.shape[1:]))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

# Set Hyperparameters
Optimizer='adam'
#Optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

# Initialize and compile:
model.compile(optimizer=Optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

After execution, we display a summary of our ANN model as shown in the cell result above.
Note the total number of weights & bias to be determined during the training sequence and understand how complex can be to train very complex ANN with hundreds of layers.
Here we have 3 layers amounting to 320 + 692352 + 1290 = 693962 parameters.

##  Step 4: Build,test & run the CNN model
Now we are ready to train our defined model against the MNIST training data.

We train the model on test set.   
The processing time will depending on the configuration of your environment.
You will see the various iterations and epochs with an accuracy & loss values associated to the model.

In [ ]:
%%time
# Train the Classifier through the ART wrapper
Original_Classifier = KerasClassifier(model)
Original_Classifier.fit(x_train, y_train, batch_size=128, nb_epochs=10)
scores = model.evaluate(x_test, y_test, verbose=1)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

##  Step 5: Analyse results
Finally let's test which digits are properly or not recognized by our generated model.   
We run `predict`ions on the images present in the test set and compare the predicted digit with the actual (annotated) one

We show the 30 first wrongly predicted images

In [ ]:
# Test the classifier against fresh test data
predOriOri = Original_Classifier.predict(x_test)
# count false predictions and display 30 first mismatches
AccOriOri = showImagesAccuracy('Showing first 30 test images that are wrongly classified by Keras original classifier, and actual->predicted',x_test, y_test, predOriOri, 30)

## Step 6: Enhancing the model with ART

Thanks to the ART toolbox we have a full set of attacks & defense available to assess our classifier.
The ART library contains implementations of the following evasion attacks:
    * DeepFool (Moosavi-Dezfooli et al., 2015)
    * Fast Gradient Method (Goodfellow et al., 2014)
    * Basic Iterative Method (Kurakin et al., 2016)
    * Projected Gradient Descent (Madry et al., 2017)
    * Jacobian Saliency Map (Papernot et al., 2016)
    * Universal Perturbation (Moosavi-Dezfooli et al., 2016)
    * Virtual Adversarial Method (Miyato et al., 2015)
    * C&W Attack (Carlini and Wagner, 2016)
    * NewtonFool (Jang et al., 2017)

We will now start challenging our Hand Digit recognition model.

We'll use one of the above implementations, the Fast Gradient Method and assess our model against it.
MNIST training dataset is made of 60000 samples so to save some time let's create only 10% of modified data from it.

We apply the `FastGradientMethod` from ART toolbox to the MNIST images and then evaluate how our model scores using the original classifier

In [ ]:
# Create modified train sub dataset (100 first images) with noise on it.
# Craft adversarial samples with the FastGradient Method
adv_crafter_FGM = FastGradientMethod(Original_Classifier, eps=0.5)

# generate one tenth of images for training set and full for test set
x_train_adv_FGM = adv_crafter_FGM.generate(x_train[:(len(x_train)//10)])
x_test_adv_FGM = adv_crafter_FGM.generate(x_test[:len(x_test)])
showImages(x_train_adv_FGM,30,15,"Example of first 30 images after applying the FastGradient perturbations")

Now that we have generated the perturbated images, let's assess our original classifier against this new training dataset

In [ ]:
# Challenge the Classifier with FastGradient modified dataset
predFGMOri = Original_Classifier.predict(x_test_adv_FGM[:len(x_test_adv_FGM)])
AccFGMOri=showImagesAccuracy('Showing first 30 perturbated images that are wrongly classified by Keras original classifier, and actual->predicted',x_test_adv_FGM, y_test, predFGMOri, 30,15)

The quality of our model goes down from above 98% accuracy to less than 8%

## Step 7: Enhancing the performance of classification
You can refer to the ART gitHub writeup to get a full sample on how to train more efficiently your classifier.

In this tutorial, we'll explore only one, so that we can observe rapidly ART's added value.

To do so let's enrich our initial training data with the one we just created for the Fast Method Gradient Attack, and then retrain the same ANN model structure

In [ ]:
# Data augmentation: expand the training set with the adversarial samples
x_train_robust = np.append(x_train, x_train_adv_FGM, axis=0)
y_train_robust = np.append(y_train, y_train[:len(x_train_adv_FGM)], axis=0)
x_test_robust = np.append(x_test, x_test_adv_FGM[:len(x_test_adv_FGM)], axis=0)
y_test_robust = np.append(y_test, y_test[:len(x_test_adv_FGM)], axis=0)

# Retrain the CNN on the extended dataset
model.compile(loss='categorical_crossentropy', optimizer=Optimizer, metrics=['accuracy'])

In [ ]:
%%time
# and train a classifier
Robust_Classifier = KerasClassifier(model)
Robust_Classifier.fit(x_train_robust, y_train_robust, nb_epochs=10, batch_size=128)

## Step 8: Check retrained classifier on modified dataset

In [ ]:
# Challenge the Robust Classifier with FastGradient modified dataset
predFGMRob = Robust_Classifier.predict(x_test_adv_FGM[:len(x_test_adv_FGM)])
AccFGMRob = showImagesAccuracy('Showing first 30 perturbated images that are wrongly classified by Keras retrained classifier, and actual->predicted',x_test_adv_FGM, y_test, predFGMRob, 30)

## Step 9: Check retrained classifier on original dataset

In [ ]:
# Test the Robust Classifier with original dataset
predOriRob = Robust_Classifier.predict(x_test)
# count false predictions and display 30 first mismatches
AccOriRob = showImagesAccuracy('Showing first 30 original images that are wrongly classified by Keras retrained classifier, and actual->predicted',x_test, y_test, predOriRob, 30)